# SuAVE Image to Label CNN Prediction Model

In [1]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
//IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 

<IPython.core.display.Javascript object>

### Import all packages

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
import numpy as np
matplotlib.use("Agg")

In [3]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
#from pyimagesearch.lenet import LeNet
from imutils import paths

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# import the necessary packages
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K

In [5]:
class LeNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)

		# first set of CONV => RELU => POOL layers
		model.add(Conv2D(64, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# second set of CONV => RELU => POOL layers
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# third set of CONV => RELU => POOL layers
		model.add(Conv2D(256, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
              
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1000))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [6]:
import matplotlib.pyplot as plt
#import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

In [7]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = 50
INIT_LR = 1e-3
BS = 64

In [8]:
# init the image suffix, yset, and image list
print("[INFO] loading images...")
suffix = '.jpg'
img_list = []
yset = []

[INFO] loading images...


In [9]:
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

In [10]:
# Testing Cells
csv_file = 'balthus_v1.csv'

In [11]:
# use csv file to grab images/labels
df = pd.read_csv(csv_file)
nameCol = df['#img']
predCol = df['Style']

In [12]:
# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])

In [13]:
# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

In [14]:
# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i

In [15]:
label_yval

{'Expressionism': 1,
 'Expressionism, Japonism': 2,
 'Expressionism, Surrealism': 6,
 'Metaphysical art': 5,
 'Metaphysical art, Surrealism': 4,
 'Realism': 3,
 'Surrealism': 0}

In [16]:
len(labels)

94

In [17]:
label_yval[labels[0]]

1

In [18]:
# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])

In [19]:
len(yset)

94

In [20]:
# remove text and leave fabric cave number for labels and zero index
#for i in range (0,len(yset)):
#    yset[i] = int(re.sub("\D", "", yset[i]))
#    yset[i] = yset[i]-1

In [21]:
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join("./images3/", base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

In [22]:
Y = []
X = []

In [23]:
# Shuffle the data
p = np.random.permutation(len(yset))

In [24]:
for i in range(0,len(yset)):
    Y.append(yset[p[i]])
    X.append(img_list[p[i]])

In [25]:
# split the test and training set 75:25
split = int(len(X)*(.75))
xtrain = X[:split]
xtest = X[split:]
ytrain = Y[:split]
ytest = Y[split:]

In [26]:
# transform to arrays
trainX = np.array(xtrain, dtype="float")/225.0
testX = np.array(xtest, dtype ="float")/225.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [27]:
trainX.shape

(70, 28, 28, 3)

In [28]:
# parsed Y data containers
trainY = []
testY = []

In [29]:
# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,12)
testY = np_utils.to_categorical(ytest,12)

In [30]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

In [31]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=12)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

[INFO] compiling model...


In [32]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/50
1/1 [==============================] - 0s 489ms/step - loss: 2.4001 - acc: 0.1667 - val_loss: 1.2565 - val_acc: 0.8750
Epoch 2/50
1/1 [==============================] - 1s 719ms/step - loss: 1.2854 - acc: 0.8281 - val_loss: 1.1544 - val_acc: 0.8750
Epoch 3/50
1/1 [==============================] - 1s 562ms/step - loss: 1.0865 - acc: 0.8438 - val_loss: 1.2200 - val_acc: 0.8750
Epoch 4/50
1/1 [==============================] - 0s 114ms/step - loss: 3.3396 - acc: 0.5000 - val_loss: 0.8612 - val_acc: 0.8750
Epoch 5/50
1/1 [==============================] - 0s 450ms/step - loss: 0.8687 - acc: 0.8125 - val_loss: 0.9284 - val_acc: 0.8750
Epoch 6/50
1/1 [==============================] - 0s 101ms/step - loss: 0.8891 - acc: 0.8333 - val_loss: 1.1222 - val_acc: 0.8750
Epoch 7/50
1/1 [==============================] - 0s 186ms/step - loss: 1.1896 - acc: 0.6667 - val_loss: 1.2080 - val_acc: 0.8750
Epoch 8/50
1/1 [==============================] - 0s 415ms/step

In [33]:
# plot the training loss and accuracy 
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Label Prediction")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('figure.png')

In [34]:
# Shows chart in a popup. Close to keep going
#chart = cv2.imread('figure.png',1)
#cv2.imshow('Results',chart)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [35]:
# Reshape original input data images for predicting
img_check = np.array(img_list, dtype ="float")/225.0

In [36]:
predictionsMade = []

In [37]:
preds = model.predict(img_check)

In [38]:
# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)

In [39]:
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 

In [40]:
print("Accuracy: " + str(correct/len(yset)))

Accuracy: 0.8297872340425532


In [41]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

In [42]:
# Append the new column
df['#predictions'] = finalPred

In [43]:
# new file name and save
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(new_file, index=None)

In [44]:
#Input survey name
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

print("Input survey name here:")
# Display input text box widget for input
display(input_text)

display(output_text)

Input survey name here:


Text(value='')

Text(value='')

In [45]:
#Print survey name
survey_name = output_text.value
print("Survey name is:", survey_name)

Survey name is: balthus_survey_analysis


In [46]:
import requests
upload_url = "http://localhost:3001/uploadCSV"
dzc_url = 'http://localhost:3001/imageserver/balthus/balthus.dzc'
upload_data = {'name': survey_name, 'dzc': dzc_url, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = "http://localhost:3001/main/file=" + user + "_" + survey_url + ".csv" + "&views=111000&view=grid"
print(url)

200 OK
http://localhost:3001/main/file=Zeppelin-V-Alt_balthus_survey_analysis.csv&views=111000&view=grid
